# 🕷️ Web Scraping de Datos con Python

En este notebook aprenderemos a extraer datos de sitios web usando BeautifulSoup y requests.


In [1]:
%pip install -q requests beautifulsoup4 lxml pandas html5lib

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías cargadas correctamente")
print(f"requests: {requests.__version__}")


Note: you may need to restart the kernel to use updated packages.
✅ Librerías cargadas correctamente
requests: 2.32.4


In [2]:
# Headers para simular un navegador
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'es-ES,es;q=0.9,en;q=0.8',
}

print("Headers configurados para simular navegador")


Headers configurados para simular navegador


## 2. Scraping de una página de ejemplo

Usaremos books.toscrape.com, una página diseñada para practicar scraping.


In [3]:
# Página de ejemplo para scraping
url = "https://books.toscrape.com/"

response = requests.get(url, headers=headers)
print(f"Status: {response.status_code}")

# Parsear HTML
soup = BeautifulSoup(response.content, 'lxml')
print(f"Título de la página: {soup.title.text}")


Status: 200
Título de la página: 
    All products | Books to Scrape - Sandbox



In [4]:
# Extraer información de libros
books = soup.find_all('article', class_='product_pod')

print(f"Libros encontrados: {len(books)}")
print("=" * 50)

# Mostrar primeros 5 libros
for i, book in enumerate(books[:5], 1):
    title = book.h3.a['title']
    price = book.find('p', class_='price_color').text
    availability = book.find('p', class_='instock availability').text.strip()
    rating = book.p['class'][1]  # Rating está en la clase CSS
    
    print(f"\n📚 Libro {i}:")
    print(f"   Título: {title}")
    print(f"   Precio: {price}")
    print(f"   Disponibilidad: {availability}")
    print(f"   Rating: {rating}")


Libros encontrados: 20

📚 Libro 1:
   Título: A Light in the Attic
   Precio: £51.77
   Disponibilidad: In stock
   Rating: Three

📚 Libro 2:
   Título: Tipping the Velvet
   Precio: £53.74
   Disponibilidad: In stock
   Rating: One

📚 Libro 3:
   Título: Soumission
   Precio: £50.10
   Disponibilidad: In stock
   Rating: One

📚 Libro 4:
   Título: Sharp Objects
   Precio: £47.82
   Disponibilidad: In stock
   Rating: Four

📚 Libro 5:
   Título: Sapiens: A Brief History of Humankind
   Precio: £54.23
   Disponibilidad: In stock
   Rating: Five


## 3. Función de scraping completa


In [5]:
def scrape_books_page(url):
    """Extrae información de libros de una página."""
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return []
    
    soup = BeautifulSoup(response.content, 'lxml')
    books = soup.find_all('article', class_='product_pod')
    
    # Mapeo de ratings
    rating_map = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
    
    data = []
    for book in books:
        rating_text = book.p['class'][1]
        data.append({
            'titulo': book.h3.a['title'],
            'precio': book.find('p', class_='price_color').text,
            'disponibilidad': book.find('p', class_='instock availability').text.strip(),
            'rating': rating_map.get(rating_text, 0),
            'url': 'https://books.toscrape.com/' + book.h3.a['href']
        })
    
    return data

# Probar la función
books_data = scrape_books_page("https://books.toscrape.com/")
print(f"✅ Extraídos {len(books_data)} libros")


✅ Extraídos 20 libros


In [6]:
# Convertir a DataFrame
df_books = pd.DataFrame(books_data)

# Limpiar precio (quitar símbolo £ y convertir a float)
df_books['precio_num'] = df_books['precio'].str.replace('£', '').astype(float)

print("📊 Dataset de libros:")
df_books.head(10)


📊 Dataset de libros:


,titulo,precio,disponibilidad,rating,url,precio_num
0,A Light in the Attic,£51.77,In stock,3,https://books.toscrape.com/catalogue/a-light-i...,51.77
1,Tipping the Velvet,£53.74,In stock,1,https://books.toscrape.com/catalogue/tipping-t...,53.74
2,Soumission,£50.10,In stock,1,https://books.toscrape.com/catalogue/soumissio...,50.10
3,Sharp Objects,£47.82,In stock,4,https://books.toscrape.com/catalogue/sharp-obj...,47.82
4,Sapiens: A Brief History of Humankind,£54.23,In stock,5,https://books.toscrape.com/catalogue/sapiens-a...,54.23
5,The Requiem Red,£22.65,In stock,1,https://books.toscrape.com/catalogue/the-requi...,22.65
6,The Dirty Little Secrets of Getting Your Dream...,£33.34,In stock,4,https://books.toscrape.com/catalogue/the-dirty...,33.34
7,The Coming Woman: A Novel Based on the Life of...,£17.93,In stock,3,https://books.toscrape.com/catalogue/the-comin...,17.93
8,The Boys in the Boat: Nine Americans and Their...,£22.60,In stock,4,https://books.toscrape.com/catalogue/the-boys-...,22.60
9,The Black Maria,£52.15,In stock,1,https://books.toscrape.com/catalogue/the-black...,52.15


In [7]:
# Estadísticas del dataset
print("📊 ESTADÍSTICAS DEL DATASET")
print("=" * 40)
print(f"Total libros: {len(df_books)}")
print(f"Precio promedio: £{df_books['precio_num'].mean():.2f}")
print(f"Precio mínimo: £{df_books['precio_num'].min():.2f}")
print(f"Precio máximo: £{df_books['precio_num'].max():.2f}")
print(f"\nDistribución de ratings:")
print(df_books['rating'].value_counts().sort_index())

# Guardar en CSV
df_books.to_csv('books_scraped.csv', index=False)
print("\n✅ Datos guardados en 'books_scraped.csv'")


📊 ESTADÍSTICAS DEL DATASET
Total libros: 20
Precio promedio: £38.05
Precio mínimo: £13.99
Precio máximo: £57.25

Distribución de ratings:
rating
1    6
2    3
3    3
4    4
5    4
Name: count, dtype: int64

✅ Datos guardados en 'books_scraped.csv'


## Resumen

En este ejercicio aprendimos:

1. ✅ Hacer requests HTTP con headers apropiados
2. ✅ Parsear HTML con BeautifulSoup
3. ✅ Extraer datos con `find()` y `find_all()`
4. ✅ Convertir datos a DataFrame
5. ✅ Exportar datos a CSV

### ⚠️ Recordatorio ético

- Siempre verifica los términos de servicio del sitio
- Respeta el archivo robots.txt
- No sobrecargues los servidores (usa delays)
- Prefiere APIs cuando estén disponibles
